In [4]:
# =========================
# IMPORTS Y CONFIGURACIÓN
# =========================
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# Directorios
NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.dirname(NOTEBOOK_DIR)
PROCESSED_DIR = os.path.join(PROJECT_ROOT, "data", "processed")
REG_DIR = os.path.join(PROCESSED_DIR, "regression")
PLOTS_DIR = os.path.join(REG_DIR, "plots")

os.makedirs(REG_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

print("Resultados de regresión se guardarán en:", REG_DIR)


Resultados de regresión se guardarán en: d:\Projects\University-Projects\AlphaTech-Analyzer\data\processed\regression


In [5]:
# =========================
# CARGAR DATOS PROCESADOS
# =========================

panel_df = pd.read_csv(
    os.path.join(PROCESSED_DIR, "tech30_panel_monthly_2018_2024.csv"),
    parse_dates=['Date']
)
agg_df = pd.read_csv(
    os.path.join(PROCESSED_DIR, "tech30_aggregated_stats_2018_2024.csv")
)

print("Panel mensual:", panel_df.shape)
print("Dataset agregado:", agg_df.shape)


Panel mensual: (2403, 6)
Dataset agregado: (30, 7)


In [6]:
# =========================
# RETORNOS DEL MERCADO
# =========================

import yfinance as yf

# Descargar QQQ (fallback si no existe variable)
market = yf.download("QQQ", start="2018-01-01", end="2024-12-31", progress=False)
price_col = "Adj Close" if "Adj Close" in market.columns else "Close"

market["MarketReturn"] = np.log(market[price_col] / market[price_col].shift(1))
market = market.resample("ME").last()[["MarketReturn"]].dropna()
market.index = market.index.strftime("%Y-%m-%d")

print("MarketReturn listo:", market.shape)


MarketReturn listo: (84, 1)


In [7]:
# =========================
# FUNCIÓN REGRESIÓN POR EMPRESA
# =========================
def estimate_company_regression(company, panel_df, market_returns):
    df_i = panel_df[panel_df["Company"] == company].copy()
    df_i["Date"] = df_i["Date"].astype(str)
    df = df_i.merge(
        market_returns,
        left_on="Date",
        right_index=True,
        how="inner"
    ).dropna(subset=["Return","MarketReturn","Volume"])

    if len(df) < 12:
        return None

    df["logVol"] = np.log(df["Volume"].replace(0,np.nan)).fillna(method='ffill').fillna(method='bfill')
    X = sm.add_constant(df[["MarketReturn","logVol"]])
    y = df["Return"]

    model = sm.OLS(y, X).fit(cov_type="HC1")
    resid = model.resid
    dw = durbin_watson(resid)
    bp = het_breuschpagan(resid, model.model.exog)

    return {
        "Company": company,
        "n_obs": len(df),
        "Intercept": float(model.params.get("const", np.nan)),
        "Coef_Market": float(model.params.get("MarketReturn", np.nan)),
        "SE_Market": float(model.bse.get("MarketReturn", np.nan)),
        "p_Market": float(model.pvalues.get("MarketReturn", np.nan)),
        "Coef_logVol": float(model.params.get("logVol", np.nan)),
        "SE_logVol": float(model.bse.get("logVol", np.nan)),
        "p_logVol": float(model.pvalues.get("logVol", np.nan)),
        "R2": float(model.rsquared),
        "DW": float(dw),
        "BP_stat": float(bp[0]),
        "BP_p": float(bp[1])
    }


In [8]:
# =========================
# PREPARAR RETORNOS DEL MERCADO (QQQ)
# =========================
import yfinance as yf
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

# Descargar mercado
market = yf.download("QQQ", start="2018-01-01", end="2024-12-31", progress=False)

# Determinar columna de precio ajustado
price_col = None
for col in ["Adj Close", "AdjClose", "Close"]:
    if col in market.columns:
        price_col = col
        break
if price_col is None:
    raise RuntimeError("No se encontró columna de precio válido en QQQ")

# Calcular retornos logarítmicos mensuales
market["MarketReturn"] = np.log(market[price_col] / market[price_col].shift(1))
market = market.resample("ME").last()[["MarketReturn"]].dropna()

# Convertir index a strings YYYY-MM-DD para merge
market.index = market.index.strftime("%Y-%m-%d")

# =========================
# FUNCION CAPM ROBUSTA
# =========================
def estimate_company_regression(company, panel_df, market_returns):
    df_i = panel_df[panel_df["Company"] == company].copy()
    df_i["Date"] = df_i["Date"].astype(str)

    # Merge con market_returns
    df = df_i.merge(
        market_returns,
        left_on="Date",
        right_index=True,
        how="inner"
    ).dropna(subset=["Return","MarketReturn","Volume"])

    if len(df) < 12:  # mínimo 1 año
        return None

    df["logVol"] = np.log(df["Volume"].replace(0,np.nan)).fillna(method='ffill').fillna(method='bfill')

    X = sm.add_constant(df[["MarketReturn","logVol"]])
    y = df["Return"]

    model = sm.OLS(y, X).fit(cov_type="HC1")
    resid = model.resid
    dw = durbin_watson(resid)
    bp = het_breuschpagan(resid, model.model.exog)

    return {
        "Company": company,
        "n_obs": len(df),
        "Intercept": float(model.params.get("const", np.nan)),
        "Coef_Market": float(model.params.get("MarketReturn", np.nan)),
        "SE_Market": float(model.bse.get("MarketReturn", np.nan)),
        "p_Market": float(model.pvalues.get("MarketReturn", np.nan)),
        "Coef_logVol": float(model.params.get("logVol", np.nan)),
        "SE_logVol": float(model.bse.get("logVol", np.nan)),
        "p_logVol": float(model.pvalues.get("logVol", np.nan)),
        "R2": float(model.rsquared),
        "DW": float(dw),
        "BP_stat": float(bp[0]),
        "BP_p": float(bp[1])
    }


In [9]:
# =========================
# REGRESIÓN CROSS-SECTION
# =========================
cross = agg_df.copy().dropna(subset=["MeanReturn","Beta","Volatility","AvgVolume"])
cross["logAvgVol"] = np.log(cross["AvgVolume"].replace(0,np.nan)).fillna(method='ffill').fillna(method='bfill')

X = sm.add_constant(cross[["Beta","Volatility","logAvgVol"]])
y = cross["MeanReturn"].astype(float)
model_cs = sm.OLS(y, X).fit(cov_type="HC1")

# Guardar resultados
cs_summary = {
    "params": model_cs.params.to_dict(),
    "bse": model_cs.bse.to_dict(),
    "pvalues": model_cs.pvalues.to_dict(),
    "R2": float(model_cs.rsquared),
    "n": int(model_cs.nobs)
}

# CSV coeficientes
pd.DataFrame({
    "param": model_cs.params.index,
    "coef": model_cs.params.values,
    "se": model_cs.bse.values,
    "pvalue": model_cs.pvalues.values
}).to_csv(os.path.join(REG_DIR,"cross_section_regression_coefficients.csv"), index=False)

# JSON summary
with open(os.path.join(REG_DIR,"cross_section_regression_summary.json"), "w") as f:
    import json
    json.dump(cs_summary,f,indent=4)

print("✅ Regresión cross-section guardada. R2:", model_cs.rsquared)
print(model_cs.summary())


✅ Regresión cross-section guardada. R2: 0.4895026269349185
                            OLS Regression Results                            
Dep. Variable:             MeanReturn   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.431
Method:                 Least Squares   F-statistic:                     26.91
Date:                Tue, 30 Dec 2025   Prob (F-statistic):           3.91e-08
Time:                        19:08:24   Log-Likelihood:                 101.08
No. Observations:                  30   AIC:                            -194.2
Df Residuals:                      26   BIC:                            -188.5
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------

C:\Users\fvab2\AppData\Local\Temp\ipykernel_18940\1269270762.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cross["logAvgVol"] = np.log(cross["AvgVolume"].replace(0,np.nan)).fillna(method='ffill').fillna(method='bfill')


In [11]:
# =========================
# EJECUTAR REGRESIÓN POR EMPRESA + GUARDAR CSV Y PLOTS (VERSIÓN CORREGIDA)
# =========================

import os 
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Asegurar carpetas
os.makedirs(REG_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

# =========================
# SOLUCIÓN CRÍTICA: APLANAR market SI TIENE MULTIINDEX
# =========================
print("Verificando estructura de market...")
print(f"Tipo de columnas de market: {type(market.columns)}")
print(f"Columnas actuales: {market.columns}")

# Si market tiene MultiIndex, aplanarlo
if isinstance(market.columns, pd.MultiIndex):
    print("⚠️ market tiene MultiIndex. Aplanando...")
    # Opción 1: Mantener solo la columna MarketReturn
    if ('MarketReturn', '') in market.columns:
        market = market[('MarketReturn', '')].copy()
    elif 'MarketReturn' in market.columns:
        market = market['MarketReturn'].copy()
    # Convertir a DataFrame si es Serie
    if isinstance(market, pd.Series):
        market = market.to_frame(name='MarketReturn')
    # Aplanar nombres de columnas
    market.columns = ['MarketReturn'] if len(market.columns) == 1 else market.columns
    print("✅ MultiIndex eliminado")

# Asegurar que el índice sea string para merge
if not all(isinstance(idx, str) for idx in market.index):
    print("⚠️ Convertiendo índice de market a string...")
    market.index = pd.to_datetime(market.index).strftime("%Y-%m-%d")

print(f"✅ Estructura final: {market.shape}, columnas: {market.columns.tolist()}")
print(f"✅ Índice tipo: {type(market.index[0]) if len(market) > 0 else 'vacío'}")
print("="*50)

# Ahora continúa con el resto del código
company_rows = []
plot_sample = 6    # cuántos plots guardar
plotted = 0

for company in sorted(panel_df["Company"].unique()):
    # Usar la función YA DEFINIDA (celda 7 o 8)
    res = estimate_company_regression(company, panel_df, market)
    if res is None:
        continue
    company_rows.append(res)

    # guardar diagnostics plots para primeras empresas
    if plotted < plot_sample:
        df_i = panel_df[panel_df["Company"] == company].copy()
        df_i["Date"] = df_i["Date"].astype(str)
        
        # Merge con market (ya debería tener índice string)
        df = df_i.merge(
            market, 
            left_on="Date", 
            right_index=True, 
            how="inner"
        ).dropna(subset=["Return", "MarketReturn", "Volume"])
        
        if df.shape[0] < 12:
            plotted += 1
            continue
        
        # Usar ffill()/bfill() en lugar de fillna(method=...)
        df["logVol"] = np.log(df["Volume"].replace(0, np.nan)).ffill().bfill()
        
        X = sm.add_constant(df[["MarketReturn", "logVol"]])
        y = df["Return"]
        model = sm.OLS(y, X).fit(cov_type="HC1")
        resid = model.resid

        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
        
        # Plot 1: Scatter + línea ajuste
        axes[0].scatter(df["MarketReturn"], df["Return"], alpha=0.6)
        xvals = np.linspace(df["MarketReturn"].min(), df["MarketReturn"].max(), 50)
        axes[0].plot(xvals, 
                    model.params.get("const", 0) + model.params.get("MarketReturn", 0) * xvals, 
                    color="red")
        axes[0].set_title(f"{company} — Return vs Market")
        axes[0].set_xlabel("MarketReturn")
        axes[0].set_ylabel("Return")
        
        # Plot 2: Residuales vs fitted
        fitted = model.fittedvalues
        axes[1].scatter(fitted, resid, alpha=0.6)
        axes[1].axhline(0, color='red', linestyle='--')
        axes[1].set_title("Residuales vs Fitted")
        
        # Plot 3: QQ-plot
        sm.qqplot(resid, line='s', ax=axes[2])
        axes[2].set_title("QQ-plot residuos")
        
        plt.tight_layout()
        plot_path = os.path.join(PLOTS_DIR, f"{company}_diagnostic.png")
        plt.savefig(plot_path, dpi=150)
        plt.close(fig)
        plotted += 1

# Guardar resultados
if company_rows:
    company_reg_df = pd.DataFrame(company_rows)
    out_path = os.path.join(REG_DIR, "company_monthly_regression_results.csv")
    company_reg_df.to_csv(out_path, index=False)
    
    print("\n" + "="*50)
    print("✅ Regresiones por empresa completadas:", company_reg_df.shape[0], "empresas")
    print("📁 CSV guardado en:", out_path)
    print("📁 Plots guardados en:", PLOTS_DIR)
    
    # Mostrar resumen estadístico rápido
    print("\n📊 RESUMEN DE BETA ESTIMADA:")
    print(f"  Media: {company_reg_df['Coef_Market'].mean():.4f}")
    print(f"  Mediana: {company_reg_df['Coef_Market'].median():.4f}")
    print(f"  Mín: {company_reg_df['Coef_Market'].min():.4f}")
    print(f"  Máx: {company_reg_df['Coef_Market'].max():.4f}")
    print(f"  R² promedio: {company_reg_df['R2'].mean():.4f}")
else:
    print("❌ No se pudieron calcular regresiones para ninguna empresa")

Verificando estructura de market...
Tipo de columnas de market: <class 'pandas.core.indexes.multi.MultiIndex'>
Columnas actuales: MultiIndex([('MarketReturn', '')],
           names=['Price', 'Ticker'])
⚠️ market tiene MultiIndex. Aplanando...
✅ MultiIndex eliminado
✅ Estructura final: (84, 1), columnas: ['MarketReturn']
✅ Índice tipo: <class 'str'>


C:\Users\fvab2\AppData\Local\Temp\ipykernel_18940\2349399716.py:48: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["logVol"] = np.log(df["Volume"].replace(0,np.nan)).fillna(method='ffill').fillna(method='bfill')
C:\Users\fvab2\AppData\Local\Temp\ipykernel_18940\2349399716.py:48: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["logVol"] = np.log(df["Volume"].replace(0,np.nan)).fillna(method='ffill').fillna(method='bfill')
C:\Users\fvab2\AppData\Local\Temp\ipykernel_18940\2349399716.py:48: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["logVol"] = np.log(df["Volume"].replace(0,np.nan)).fillna(method='ffill').fillna(method='bfill')
C:\Users\fvab2\AppData\Local\Temp\ipykernel_18940\2349399716.py:48: FutureWarning: Series.f


✅ Regresiones por empresa completadas: 30 empresas
📁 CSV guardado en: d:\Projects\University-Projects\AlphaTech-Analyzer\data\processed\regression\company_monthly_regression_results.csv
📁 Plots guardados en: d:\Projects\University-Projects\AlphaTech-Analyzer\data\processed\regression\plots

📊 RESUMEN DE BETA ESTIMADA:
  Media: 1.4332
  Mediana: 1.4091
  Mín: -0.6622
  Máx: 2.8205
  R² promedio: 0.0770


C:\Users\fvab2\AppData\Local\Temp\ipykernel_18940\2349399716.py:48: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["logVol"] = np.log(df["Volume"].replace(0,np.nan)).fillna(method='ffill').fillna(method='bfill')
C:\Users\fvab2\AppData\Local\Temp\ipykernel_18940\2349399716.py:48: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["logVol"] = np.log(df["Volume"].replace(0,np.nan)).fillna(method='ffill').fillna(method='bfill')
C:\Users\fvab2\AppData\Local\Temp\ipykernel_18940\2349399716.py:48: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["logVol"] = np.log(df["Volume"].replace(0,np.nan)).fillna(method='ffill').fillna(method='bfill')
C:\Users\fvab2\AppData\Local\Temp\ipykernel_18940\2349399716.py:48: FutureWarning: Series.f